# Hyperparameter Tuning

Scikit has many approaches to optimizing or tuning the hyperparameters of models. Let's take a look at how we can use `GridSearchCV` to search over a space of possible hyperparamter combinations.

## Create data

Let's create a dummy binary classification dataset.

In [1]:
import numpy as np
from sklearn.datasets import make_classification

np.random.seed(37)

X, y = make_classification(**{
    'n_samples': 2000,
    'n_features': 20,
    'n_informative': 2,
    'n_redundant': 2,
    'n_repeated': 0,
    'n_classes': 2,
    'n_clusters_per_class': 2,
    'random_state': 37
})

print(f'X shape = {X.shape}, y shape {y.shape}')

X shape = (2000, 20), y shape (2000,)


## Tuning Logistic Regression

Let's try to tune a logistic regression model. The logistic regression model will be referred to as the `estimator`; it is this estimator's possible hyperparamters that we want to optimize. When tuning hyperparameters, we also need a way to split the data, and here, we will use `StratifiedKFold`. Another important input to the grid search is the `param_grid` argument, which is a dictionary specifying the search space of each hyperparameter. Here, our search space is simple, it is over the `regularization strength`. Lastly, we need an optimization criteria, and we specify that through the [scoring argument](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold

p = {
    'solver': 'sag',
    'penalty': 'l2',
    'random_state': 37,
    'max_iter': 100
}
estimator = LogisticRegression(**p)

p = {
    'n_splits': 5,
    'shuffle': True,
    'random_state': 37
}
cv = StratifiedKFold(**p)

p = {
    'estimator': estimator,
    'cv': cv,
    'param_grid': {
        'C': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    },
    'scoring': {
        'auc': 'roc_auc',
        'apr': 'average_precision'
    },
    'verbose': 5,
    'refit': 'auc',
    'error_score': np.NaN,
    'n_jobs': -1
}
model = GridSearchCV(**p)

model.fit(X, y)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  52 out of  55 | elapsed:    1.5s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:    1.5s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=37, shuffle=True),
             estimator=LogisticRegression(random_state=37, solver='sag'),
             n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8,
                               0.9, 1.0]},
             refit='auc',
             scoring={'apr': 'average_precision', 'auc': 'roc_auc'}, verbose=5)

The `best_params_` property gives the best combination of hyperparameters.

In [3]:
model.best_params_

{'C': 0.4}

The `best_score_` property gives the best score.

In [4]:
model.best_score_

0.9644498503712592

To retrieve the best estimator induced by the search and scoring criteria, access `best_estimator_`.

In [5]:
model.best_estimator_

LogisticRegression(C=0.4, random_state=37, solver='sag')

## Tuning Random Forest

Here, we tune a `RandomForestClassifier`.

In [6]:
from sklearn.ensemble import RandomForestClassifier

p = {
    'random_state': 37
}
estimator = RandomForestClassifier(**p)

p = {
    'n_splits': 5,
    'shuffle': True,
    'random_state': 37
}
cv = StratifiedKFold(**p)

p = {
    'estimator': estimator,
    'cv': cv,
    'param_grid': {
        'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
        'criterion': ['gini', 'entropy']
    },
    'scoring': {
        'auc': 'roc_auc',
        'apr': 'average_precision'
    },
    'verbose': 5,
    'refit': 'auc',
    'error_score': np.NaN,
    'n_jobs': -1
}
model = GridSearchCV(**p)

model.fit(X, y)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    7.5s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=37, shuffle=True),
             estimator=RandomForestClassifier(random_state=37), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                          100]},
             refit='auc',
             scoring={'apr': 'average_precision', 'auc': 'roc_auc'}, verbose=5)

In [7]:
model.best_params_

{'criterion': 'entropy', 'n_estimators': 50}

In [8]:
model.best_score_

0.9763199132478311

In [9]:
model.best_estimator_

RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=37)